In [1]:
from process_traindata import lset_to_explodeddf, load_cands, create_sortedbatch_data
from new_mt_train import XLMCometRegressorAvg, XLMCometRegressor
from mt_scores import get_scores_auto
import torch
import pandas as pd
from os.path import exists
import baseline_cands as bc
#from rerank_data import rerank_df, rerank_nogold, rerank_single

2022-12-05 10:41:57.529243: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-05 10:41:57.529265: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# STEPS TO GENERATING COMPLETE SET OF TABLES - 

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
# 1. Get a distilled causal model
BESTMODS = {
    'en_de_comet': 'derlcausalfine45.pt',
    'fr_en_comet': 'maskedcont4.pt',
    'fr_en_bleurt': 'frenbleurtfine5.pt'
}
SETTING = 'en_de_comet'

In [8]:
if SETTING == 'fr_en_comet':
    model = XLMCometRegressor(drop_rate=0.1).to(device)
else:
    model = XLMCometRegressorAvg(drop_rate=0.1).to(device)
model.load_state_dict(torch.load('torchsaved/'+BESTMODS[SETTING]))
model.eval()
torch.cuda.empty_cache()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# 2. Have a test set with post-hoc, distilled model score, gold score, 32 samples per example

# ASSUME that we'vePAIRalled create_sortedbatch_data for our data
def load_dset_existing(pairname, goldsco):
    datacsv = None
    if goldsco== "comet":
        datacsv = pd.read_csv("torchsaved/cometen_detrain.csv")
        #extracsv = pd.read_csv("torchsaved/cqeen_detrain.csv")
        # since we have for comet and for cqe
        datacsv['comet'] = datacsv['score']
        #datacsv['cqe'] = extracsv['score']
    elif goldsco == "bleurt":
        datacsv = pd.read_csv("torchsaved/bleurten_detrain.csv")
    
    return datacsv.drop(columns=['score', 'Unnamed: 0'])
        
TRAINSET_NAMES = {
    'en_de_comet': 'cometen_detrain.csv',
    'fr_en_comet': 'cometen_detrain.pt',
    'fr_en_bleurt': 'frenbleurtfine5.pt'
}

PAIR = 'en_de'
SCORER = 'comet'
SPLIT = .9

if exists("torchsaved/testdata"+PAIR+SCORER+".csv"):
    testset = pd.read_csv("torchsaved/testdata"+PAIR+SCORER+".csv")
else:

    testset = load_dset_existing(PAIR, SCORER)
    testset = testset.iloc[int(len(testset)*SPLIT/32)*32:]

    hyps, srcs, refs = list(testset['hyp']), list(testset['src']), list(testset['ref'])
    # get cqe, post-hoc score, distilled model score (already have gold)
    with torch.no_grad():
        utmod = get_scores_auto(hyps, srcs, refs, "custom", model)
        cqe = get_scores_auto(hyps, srcs, refs, "cqe", "")
        posthoc = get_scores_auto(hyps, srcs, refs, "posthoc", PAIR)

    testset['utscore'] = utmod
    testset['cqe'] = cqe
    testset['posthoc'] = posthoc
    testset.to_csv("torchsaved/testdata"+PAIR+SCORER+".csv")

In [2]:
import numpy as np
import random

In [3]:
# rerank batch based on some metric
def rerank_single(selected, metric, target):
    if metric=="posthoc":
        mbind = np.argmin(selected[metric])
    else:
        mbind = np.argmax(selected[metric])
    return list(selected[target])[mbind]

# rerank single but ignore last thing in each batch
def rerank_nogold(selected, metric, target):
    if metric=="posthoc":
        mbind = np.argmin(list(selected[metric])[:-1])
    else:
        mbind = np.argmax(list(selected[metric])[:-1])
    return list(selected[target])[mbind]

# rerank single but ignore last thing in each batch
def rerank_nogweight(selected, metric, target):
    met = list(selected[metric])
    phoc = list(selected['posthoc'])
    newvec = [met[i]*.9-phoc[i]*1.25 for i in range(len(met))]
    mbind = np.argmax(newvec[:-1])
    return list(selected[target])[mbind]

# TODO changes for with / without gold
def rerank_weighted(selected, metric, target):
    met = list(selected[metric])
    phoc = list(selected['posthoc'])
    newvec = [met[i]*.9-phoc[i]*1.25 for i in range(len(met))]
    mbind = np.argmax(newvec)
    return list(selected[target])[mbind]

def rerank_rand(selected, metric, target):
    return random.choice(list(selected[target])[:-1])

def rerank_df(df, scofunct, scoparam):
    reflist = list(df['ref'].unique())
    rrdf = []
    for r in reflist:
        # extract dataframe corresponding to smth
        exsamps = df[df['ref']==r]
        rrdf.append(scofunct(exsamps, scoparam[0], scoparam[1]))
    return sum(rrdf)/len(rrdf)

In [4]:
#bcdf50 = pd.read_csv("torchsaved/bc50en_decomet.csv")
bcdf50 = pd.read_csv("testbeam50fr_en.csv")

In [55]:
#latencheck = pd.read_csv("torchsaved/testdataen_decomet.csv")
frtest = pd.read_csv("torchsaved/testdatafr_encomet.csv")

In [6]:
bcdf50.keys()

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3',
       'level_0', 'Unnamed: 0.2', 'index', 'Unnamed: 0.1', 'Unnamed: 0', 'src',
       'hyp', 'ref', 'utscore', 'cqe', 'posthoc', 'comet', 'dupcqe'],
      dtype='object')

In [11]:
rerank_df(bcdf50, rerank_weighted, ['cqe', 'comet'])

0.6923318155736187

In [10]:
# 3. For a smaller set, get exploded stuff and run stuff from above on that
MINISET = 100
exploded_dirs = {
    "en_de":"./candoutputs/explodedmt1n_en-de_bfs_recom_4_80_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl",
    "fr_en":"./candoutputs/explodedmtn1_fr-en_bfs_recom_4_-1_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl",
    "en_deb1":"./candoutputs/testbeam1en_de.jsonl",
    "en_deb50":"./candoutputs/testbeam50en_de.jsonl"
}

In [ ]:
if exists("torchsaved/exlpodedmini"+PAIR+SCORER+".csv"):
    expdf = pd.read_csv("torchsaved/exlpodedmini"+PAIR+SCORER+".csv")
else:
    lcands = load_cands(exploded_dirs[PAIR])
    lcands = lcands[-100:]
    len(lcands)
    expdf = lset_to_explodeddf(lcands)

    hyps, srcs, refs = list(expdf['hyp']), list(expdf['src']), list(expdf['ref'])
    # get cqe, post-hoc score, distilled model score (already have gold)
    with torch.no_grad():
        utmode = get_scores_auto(hyps, srcs, refs, "custom", model)
        golde = get_scores_auto(hyps, srcs, refs, "comet", "")
        cqee = get_scores_auto(hyps, srcs, refs, "cqe", "")
        posthoce = get_scores_auto(hyps, srcs, refs, "posthoc", PAIR)

    expdf['utscore'] = utmode
    expdf['cqe'] = cqee
    expdf['posthoc'] = posthoce
    expdf['comet'] = golde

    expdf.to_csv("torchsaved/exlpodedmini"+PAIR+SCORER+".csv")

In [24]:
len(testset['src'].unique())

1050

In [18]:
del model

In [ ]:
import baseline_cands as bc
# 4. Get beam search 1 and beam search 50 for these
if exists("./candoutputs/testbeam"+str(50)+PAIR+".jsonl"):
    print("already generated")
else:
    btok, bmodel = bc.load_model(PAIR, "cuda:2")
    bmodel.eval()
    with torch.no_grad():
        bscands = bc.get_generate_candidates(bc.beam_generate, list(testset['src'].unique()), list(testset['ref'].unique())
                    , btok, bmodel, 50, 1, PAIR, "cuda:2", "beam", 80)
        bscands = bc.get_generate_candidates(bc.beam_generate, list(testset['src'].unique()), list(testset['ref'].unique())
                    , btok, bmodel, 1, 1, PAIR, "cuda:2", "beam", 80)

In [9]:
def generate_scores(inpdf):
    hyps, srcs, refs = list(inpdf['hyp']), list(inpdf['src']), list(inpdf['ref'])
    # get cqe, post-hoc score, distilled model score (already have gold)
    with torch.no_grad():
        utmode = get_scores_auto(hyps, srcs, refs, "custom", model)
        golde = get_scores_auto(hyps, srcs, refs, "comet", "")
        cqee = get_scores_auto(hyps, srcs, refs, "cqe", "")
        posthoce = get_scores_auto(hyps, srcs, refs, "posthoc", PAIR)

    inpdf['utscore'] = utmode
    inpdf['cqe'] = cqee
    inpdf['posthoc'] = posthoce
    inpdf['comet'] = golde
    return inpdf

# 4.5 Generate appropriate scores for these candidates
bcdf1 = lset_to_explodeddf(load_cands(exploded_dirs[PAIR+"b1"]))
bcdf50 = lset_to_explodeddf(load_cands(exploded_dirs[PAIR+"b50"]))
# generate suite of scores
bcdf50 = generate_scores(bcdf50)

In [10]:
bcdf50.to_csv("torchsaved/bc50"+PAIR+SCORER+".csv")
bcdf1 = generate_scores(bcdf1)
bcdf1.to_csv("torchsaved/bc1"+PAIR+SCORER+".csv")

rescoring candidates
0
500
1000
TOTAL TIME  16.95


In [ ]:
# 5. Run lattice pipeline on the whole thing, evaluate generated options

# TODO for german pinpoint the right lattices